# Test1 LGBM and ExtraTrees with or without PCA

In [ ]:
import numpy as np
import sklearn
import pandas as pd
from lightgbm import LGBMClassifier
import warnings

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import PCA

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

In [ ]:
path = '/home/fds/Dev/Python/2022_AI_competition/data/'

In [ ]:
train  = pd.read_csv(path+'train.csv')
test  = pd.read_csv(path+'test.csv')

In [ ]:
# drop_list = ['Q1','Q2','Q3','Q4','Q5', 'Q6','Q7','Q8','Q9',
#             'Q10','Q11','Q12','Q13','Q14','Q15', 'Q16','Q17','Q18','Q19',
#             'Q20','Q21','Q22','Q23','Q24','Q25', 'Q26',
#             'index', 'country', 'introelapse','testelapse','surveyelapse']
drop_list = ['index', 'country', ]
# replace_dict = {'education': str, 'married': str}
replace_dict = {'education': str, 'engnat': str, 'married': str, 'urban': str}
# replace_dict = {'education': str, 'engnat': str, 'urban': str}
train_data =pd.read_csv(path+'train.csv')
hold_out = int(len(train_data)*0.8)
test_data = pd.read_csv(path+'test.csv')

# train_data = train_data.drop(train_data[train_data.familysize > 11].index)

# train = train_data.drop(drop_list, axis=1)
train_y = train_data['nerdiness']
train_x = train_data.drop(drop_list + ['nerdiness'], axis=1)
test_x = test_data.drop(drop_list, axis=1)

train_x = train_x.reset_index(drop=True)
train_y = train_y.reset_index(drop=True)


columns = train_x.columns.values
print(columns)

train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)

imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mode.fit(train_x)
train_x = imp_mode.transform(train_x)
test_x = imp_mode.transform(test_x)

# scaler = RobustScaler()
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(train_x)
X_test_scaled = scaler.transform(test_x)

# scaler.fit(train_x)
# X_train_scaled = scaler.transform(train_x)
# X_test_scaled = scaler.transform(test_x)

# train = np.nan_to_num(train)
# test = np.nan_to_num(test)

# X_val_scaled = X_train_scaled[hold_out:]
# X_train_scaled = X_train_scaled[:hold_out]
# y_val = train_y[hold_out:]
# y_train = train_y[:hold_out]

# val = train[hold_out:]
# train = train[:hold_out]

# X_train_scaled, train_y_t = torch.tensor(X_train_scaled.reshape(-1, 2, 200)).to(DEVICE), torch.tensor(train_y).to(DEVICE)  
# X_test_scaled = torch.tensor(X_train_scaled.reshape(-1, 2, 200)).to(DEVICE)
pca = PCA(n_components=62) # 주성분을 몇개로 할지 결정
X_train_scaled_printcipalComponents = pca.fit_transform(X_train_scaled)
X_test_scaled_printcipalComponents = pca.transform(X_test_scaled)

X_val_scaled = X_train_scaled_printcipalComponents[hold_out:]
X_train_scaled = X_train_scaled_printcipalComponents[:hold_out]
y_val = train_y[hold_out:]
y_train = train_y[:hold_out]

# principalDf = pd.DataFrame(data=printcipalComponents, columns = columns[0:62])
print(X_train_scaled_printcipalComponents.shape)

# train_y_t = torch.tensor(train_y, dtype=torch.float32)
# X_train_scaled = torch.tensor(X_train_scaled_printcipalComponents, dtype=torch.float32)
# X_test_scaled = torch.tensor(X_test_scaled_printcipalComponents, dtype=torch.float32)

test_len = len(test_x)

N_REPEAT = 5
N_SKFOLD = 7
N_EPOCH = 300
BATCH_SIZE = 2048
VIRTUAL_BATCH_SIZE = 1024
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}

prediction = np.zeros((test_len, 1), dtype=np.float32)


In [ ]:
train.head()

In [ ]:
train.info()

## Preprocessing

In [ ]:
train = train.drop(['index', 'country'],axis = 1)
test = test.drop(['index', 'country'],axis = 1)


In [ ]:
#train을 target과 feature로 나눠줍니다.
train_x=train.drop(['nerdiness'], axis=1)
train_y=train['nerdiness']

In [ ]:
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mode.fit(train_x)
train_x = imp_mode.transform(train_x)
test_x = imp_mode.transform(test)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.33, random_state=123)

In [ ]:
(train_x == np.NaN).sum()

## Modeling

In [ ]:
# lgbm_clf = LGBMClassifier(
#             n_estimators=1000
#         )
# lgbm_clf.fit(X_train, y_train)

# lgbm_pred = lgbm_clf.predict(test)

In [ ]:
et_clf = ExtraTreesClassifier(n_estimators=20000, criterion="entropy",max_depth=None, min_samples_split=10, random_state=123, n_jobs=-1)


In [ ]:
et_clf.fit(X_train_scaled, y_train)

In [ ]:
roc_auc_score(y_val, et_clf.predict_proba(X_val_scaled)[:,1])

In [ ]:
proba = et_clf.predict_proba(X_val)[:,1]

In [ ]:
proba

In [ ]:
scores = cross_val_score(et_clf, train_x, train_y, scoring = 'accuracy', cv =5)
print(scores)

## Submission

In [ ]:
proba = et_clf.predict(test_x)

In [ ]:
submission = pd.read_csv(path+'sample_submission.csv')
submission["nerdiness"] = proba
submission.to_csv("submission.csv", index = False)
submission

# Test2 AutoML

In [ ]:
from pycaret.classification import *

In [ ]:
train  = pd.read_csv(path+'train.csv')
test  = pd.read_csv(path+'test.csv')
train = train.drop(['index', 'country'],axis = 1)
test = test.drop(['index', 'country'],axis = 1)

## Setup(Preprocessing)

In [ ]:
clf = setup(data=train, target='nerdiness', 
            feature_selection = True,
            pca = True, pca_components = 10,
            imputation_type='iterative',
            remove_multicollinearity = True,
            multicollinearity_threshold = 0.95, 
            fix_imbalance = True,
            fold_strategy = 'stratifiedkfold', fold = 10, fold_shuffle=True, 
            session_id = 123, use_gpu=True)

## Train

In [ ]:
best_5_l = compare_models(include=['et','rf','xgboost','lightgbm'], sort='AUC', n_select=2)

In [ ]:
blended = blend_models(estimator_list = best_5_l, fold = 10, method = 'soft')

In [ ]:
pred_holdout = predict_model(blended)

In [ ]:
final_model = finalize_model(blended)

In [ ]:
predictions = predict_model(final_model, data = test)
predictions

## Submission

In [ ]:
submission = pd.read_csv(path+'sample_submission.csv')
submission["nerdiness"] = predictions['Label']
submission["nerdiness"]

In [ ]:
submission.to_csv('submission_aml_label.csv', index = False)

In [ ]:
best = pd.read_csv('/home/fds/Dev/Python/2022_AI_competition/data/submission_inverse_conv1d_pred_fold7_1.csv')
compare1 = pd.read_csv('/home/fds/Dev/Python/2022_AI_competition/submission_model_PCA_inverse_deeper2_repeat3_fold6_0.0_load.csv')
compare2 = pd.read_csv('/home/fds/Dev/Python/2022_AI_competition/submission_model_PCA_inverse_deeper2_repeat4_fold6_0.0_load.csv')
compare3 = pd.read_csv('/home/fds/Dev/Python/2022_AI_competition/submission_model_PCA_inverse_deeper2_repeat2_fold6_2.0146200902141807e-10_load.csv')
lastoutput1 = pd.read_csv('/home/fds/Dev/Python/2022_AI_competition/data/submission_model_PCA_inverse_deeper2_conv1d_pred_fold7_go_0?_.csv')
lastoutput2 = pd.read_csv('/home/fds/Dev/Python/2022_AI_competition/data/submission_model_PCA_inverse_deeper2_conv1d_pred_fold7_go_1.csv')


In [ ]:
best = best['nerdiness'] > 0
compare1 = compare1['nerdiness'] > 0
compare2 = compare2['nerdiness'] > 0
compare3 = compare3['nerdiness'] > 0
lastoutput1 = lastoutput1['nerdiness'] > 0
lastoutput2 = lastoutput2['nerdiness'] > 0

In [ ]:
best.compare(compare1)

In [ ]:
best.compare(lastoutput2)


In [ ]:
lastoutput2.compare(lastoutput1)

In [ ]:
best.compare(compare1).sum()

# Test3 Neural network with conv1d (But it has mistakes in train code!)

## Import

In [ ]:

import sklearn
import random
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer

from sklearn.metrics import roc_auc_score

from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from tqdm import tqdm
import seaborn as sns

from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

import warnings
warnings.filterwarnings('ignore')

# Fixed seed for reproducibility
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

### Path

In [ ]:
path = '/home/fds/Dev/Python/2022_AI_competition/data/'
save_path = '/data/pytorch/2022_AI_Competition/model/'

## Preprocessing

In [ ]:
drop_list = ['index', 'country', 'introelapse','testelapse','surveyelapse', 'hand']
replace_dict = {'education': str, 'engnat': str, 'married': str, 'urban': str}
train_data =pd.read_csv(path+'train.csv')
test_data = pd.read_csv(path+'test.csv')

train_data = train_data.drop(train_data[train_data.familysize > 11].index)

train_y = train_data['nerdiness']
train_x = train_data.drop(drop_list + ['nerdiness'], axis=1)
test_x = test_data.drop(drop_list, axis=1)

train_x = train_x.reset_index(drop=True)
train_y = train_y.reset_index(drop=True)


columns = train_x.columns.values

train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)

imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mode.fit(train_x)
train_x = imp_mode.transform(train_x)
test_x = imp_mode.transform(test_x)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(train_x)
X_test_scaled = scaler.transform(test_x)

train_y_t = torch.tensor(train_y, dtype=torch.float32)
X_train_scaled = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_scaled = torch.tensor(X_test_scaled, dtype=torch.float32)

test_len = len(test_x)

N_REPEAT = 5
N_SKFOLD = 7
N_EPOCH = 100
BATCH_SIZE = 2048
VIRTUAL_BATCH_SIZE = 1024
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}

prediction = np.zeros((test_len, 1), dtype=np.float32)

## Model

In [ ]:
# https://www.kaggle.com/code/nawidsayed/lightgbm-and-cnn-3rd-place-solution/notebook
# https://www.kaggle.com/code/nagiss/9-solution-nagiss-part-2-2-weight-sharing-nn/notebook

# class Conv1d_with_SEBlock(nn.Module):
#     def __init__(self) -> None:
#         super().__init__()
#         pass

class Conv1dBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels) -> None:
        super(Conv1dBlock, self).__init__()
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.activation = nn.LeakyReLU()
        self.batchNorm1d = nn.BatchNorm1d(out_channels)

    def forward(self, x):
        x = self.conv1d(x)
        x = self.batchNorm1d(x)
        x = self.activation(x)
        return x


class SangModel(nn.Module):
    def __init__(self):
        super(SangModel, self).__init__()
        self.hidden_size = 64

        self.conv7 = Conv1dBlock(63, self.hidden_size*64)  
        self.conv6 = Conv1dBlock(self.hidden_size*64, self.hidden_size*32)  
        self.conv5 = Conv1dBlock(self.hidden_size*32, self.hidden_size*16)  
        self.conv4 = Conv1dBlock(self.hidden_size*16, self.hidden_size*8)  
        self.conv3 = Conv1dBlock(self.hidden_size*8, self.hidden_size*4)  
        self.conv2 = Conv1dBlock(self.hidden_size*4, self.hidden_size*2)  
        self.conv1 = Conv1dBlock(self.hidden_size*2, self.hidden_size)  

        self.fc = nn.Linear(self.hidden_size, 1)
        
    def forward(self, x_):
        x = x_.unsqueeze(-1)

        x = self.conv7(x)
        x = self.conv6(x)
        x = self.conv5(x)
        x = self.conv4(x)
        x = self.conv3(x)
        x = self.conv2(x)
        x = self.conv1(x)

        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x    

model = SangModel()

In [ ]:
for repeat in range(N_REPEAT):

    skf, tot = StratifiedKFold(n_splits=N_SKFOLD, random_state=repeat, shuffle=True), 0.
    for skfold, (train_idx, valid_idx) in enumerate(skf.split(X_train_scaled, train_y_t)):
        train_idx, valid_idx = list(train_idx), list(valid_idx)

        train_loader = DataLoader(TensorDataset(X_train_scaled[train_idx, :], train_y_t[train_idx]),
                                  shuffle=True, drop_last=True, **LOADER_PARAM)
        valid_loader = DataLoader(TensorDataset(X_train_scaled[valid_idx, :], train_y_t[valid_idx]),
                                  shuffle=False, drop_last=False, **LOADER_PARAM)
        test_loader = DataLoader(TensorDataset(X_test_scaled, torch.zeros((test_len,), dtype=torch.float32)),
                                 shuffle=False, drop_last=False, **LOADER_PARAM)
        
        # model = Model() # Problem! We miss that code.  
        model.to(DEVICE)
        criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
        optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=7.8e-2)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.99)

        prediction_t, loss_t, acc_t = np.zeros((test_len, 1), dtype=np.float32), 1., 0.

        # for epoch in range(N_EPOCH):
        for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(skfold + 1, N_SKFOLD)):
            model.train()
            train_running_acc, train_running_loss, train_running_count= 0, 0., 0
            for idx, (xx, yy) in enumerate(train_loader):
                optimizer.zero_grad()
                xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                pred = model(xx).squeeze()
                loss = criterion(pred, yy)
                train_running_loss += loss.item() * len(yy)
                train_running_count += len(yy)
                train_running_acc += ((torch.sigmoid(pred) > 0.5).float() == yy).sum().item()

                loss.backward()
                optimizer.step()
                scheduler.step()
            if epoch % 10 == 0:
                print(f"Train loss: {train_running_loss / train_running_count}")
                print(f"Train accuracy: {train_running_acc / train_running_count}")
                
            with torch.no_grad():
                model.eval()
                running_acc, running_loss, running_count = 0, 0., 0
                for xx, yy in valid_loader:
                    xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                    pred = model(xx).squeeze()
                    loss = criterion(pred, yy)
                    running_loss += loss.item() * len(yy)
                    running_count += len(yy)
                    running_acc += ((torch.sigmoid(pred) > 0.5).float() == yy).sum().item()

                if epoch % 10 == 0:
                    print(f"Valid loss: {running_loss / running_count}")
                    print(f"Valid accuracy: {running_acc / running_count}")
                # if running_acc / running_count > acc_t:
                if running_loss / running_count < loss_t:
                    loss_t = running_loss / running_count
                    acc_t = running_acc / running_count
                    torch.save(model.state_dict(), f"{save_path}model_inverse_deep1_repeat{repeat}_fold{skfold}_{loss_t}.pth")
                    for idx, (xx, _) in enumerate(test_loader):
                        xx = xx.to(DEVICE)
                        # pred = (torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
                        pred = model(xx).detach().to('cpu').numpy()
                        prediction_t[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] \
                            = pred[:, :].copy()
        prediction[:, :] += prediction_t[:, :].copy() / (N_REPEAT * N_SKFOLD)
        tot += loss_t
    print('R{} -> {:6.10f}'.format(repeat + 1, tot / N_SKFOLD))

df = pd.read_csv(path+'sample_submission.csv')
df.iloc[:, 1:] = prediction

## Submission

### Using prediction in train function

In [ ]:
df.to_csv(path+'model_inverse_batchNorm_sigmoid_repeat_2.csv', index=False)

### Load and predict

In [ ]:
# model = SangModel()
model.load_state_dict(torch.load(save_path+"model_inverse_deeper1_1_repeat0_fold2_3.762843291348371e-06.pth"))
# model.to('cpu')
model.eval()

In [ ]:
# X_test_scaled = X_test_scaled.to('cpu')
# pred = model(X_test_scaled).detach().cpu().numpy()
pred = model(X_test_scaled).detach().numpy()

In [ ]:
pred

In [ ]:
submission = pd.read_csv(path+'sample_submission.csv')
submission["nerdiness"] = pred
submission.to_csv("submission_model_inverse_deeper1_1_repeat0_fold2_3.762843291348371e-06.csv", index = False)
submission

## Summary
- Although trian function mistakes lead to overfitting, it made the model reach higher AUC scores. (I judged that lowest loss was a best model to make high AUC scores. Thank to loss metrics, I got a best model of all my model.)

# Test4 Neural network with fixed train function mistakes (After competition)

## Import

In [ ]:
import random
from datetime import datetime

import matplotlib.pyplot as plt

import numpy as np
import sklearn
import pandas as pd
import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer

from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score

from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from tqdm import tqdm
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Fixed seed for reproducibility
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
# torch.cuda.empty_cache() 

In [ ]:
path = '/home/fds/Dev/Python/2022_AI_competition/data/'
save_path = '/data/pytorch/2022_AI_Competition/model/'

## Preprocessing

### Basic
 (If you want to use PCA data, do not execute codes under this line.)

In [ ]:
# drop_list = ['Q1','Q2','Q3','Q4','Q5', 'Q6','Q7','Q8','Q9',
#             'Q10','Q11','Q12','Q13','Q14','Q15', 'Q16','Q17','Q18','Q19',
#             'Q20','Q21','Q22','Q23','Q24','Q25', 'Q26',
#             'index', 'country', 'introelapse','testelapse','surveyelapse']
drop_list = ['index', 'country', 'introelapse','testelapse','surveyelapse', 'hand']
# replace_dict = {'education': str, 'married': str}
replace_dict = {'education': str, 'engnat': str, 'married': str, 'urban': str}
train_data =pd.read_csv(path+'train.csv')
hold_out = int(len(train_data)*0.8)
test_data = pd.read_csv(path+'test.csv')

train_data = train_data.drop(train_data[train_data.familysize > 11].index)

# train = train_data.drop(drop_list, axis=1)
train_y = train_data['nerdiness']
train_x = train_data.drop(drop_list + ['nerdiness'], axis=1)
test_x = test_data.drop(drop_list, axis=1)

train_x = train_x.reset_index(drop=True)
train_y = train_y.reset_index(drop=True)


columns = train_x.columns.values
print(columns)

train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)

imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mode.fit(train_x)
train_x = imp_mode.transform(train_x)
test_x = imp_mode.transform(test_x)

# scaler = RobustScaler()
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(train_x)
X_test_scaled = scaler.transform(test_x)

# scaler.fit(train_x)
# X_train_scaled = scaler.transform(train_x)
# X_test_scaled = scaler.transform(test_x)

# train = np.nan_to_num(train)
# test = np.nan_to_num(test)

# X_val_scaled = X_train_scaled[hold_out:]
# X_train_scaled = X_train_scaled[:hold_out]
# y_val = train_y[hold_out:]
# y_train = train_y[:hold_out]

# val = train[hold_out:]
# train = train[:hold_out]

# X_train_scaled, train_y_t = torch.tensor(X_train_scaled.reshape(-1, 2, 200)).to(DEVICE), torch.tensor(train_y).to(DEVICE)  
# X_test_scaled = torch.tensor(X_train_scaled.reshape(-1, 2, 200)).to(DEVICE)
train_y_t = torch.tensor(train_y, dtype=torch.float32)
X_train_scaled = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_scaled = torch.tensor(X_test_scaled, dtype=torch.float32)

input_shape = X_train_scaled.shape[1]
test_len = len(test_x)

N_REPEAT = 5
N_SKFOLD = 7
N_EPOCH = 1000
BATCH_SIZE = 2048
VIRTUAL_BATCH_SIZE = 1024
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}

prediction = np.zeros((test_len, 1), dtype=np.float32)


### PCA

In [ ]:
# drop_list = ['Q1','Q2','Q3','Q4','Q5', 'Q6','Q7','Q8','Q9',
#             'Q10','Q11','Q12','Q13','Q14','Q15', 'Q16','Q17','Q18','Q19',
#             'Q20','Q21','Q22','Q23','Q24','Q25', 'Q26',
#             'index', 'country', 'introelapse','testelapse','surveyelapse']
drop_list = ['index', 'country', ]
# replace_dict = {'education': str, 'married': str}
replace_dict = {'education': str, 'engnat': str, 'married': str, 'urban': str}
# replace_dict = {'education': str, 'engnat': str, 'urban': str}
train_data =pd.read_csv(path+'train.csv')
hold_out = int(len(train_data)*0.8)
test_data = pd.read_csv(path+'test.csv')

# train_data = train_data.drop(train_data[train_data.familysize > 11].index)

# train = train_data.drop(drop_list, axis=1)
train_y = train_data['nerdiness']
train_x = train_data.drop(drop_list + ['nerdiness'], axis=1)
test_x = test_data.drop(drop_list, axis=1)

train_x = train_x.reset_index(drop=True)
train_y = train_y.reset_index(drop=True)


columns = train_x.columns.values
print(columns)

train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)

imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mode.fit(train_x)
train_x = imp_mode.transform(train_x)
test_x = imp_mode.transform(test_x)

# scaler = RobustScaler()
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(train_x)
X_test_scaled = scaler.transform(test_x)

# scaler.fit(train_x)
# X_train_scaled = scaler.transform(train_x)
# X_test_scaled = scaler.transform(test_x)

# train = np.nan_to_num(train)
# test = np.nan_to_num(test)

# X_val_scaled = X_train_scaled[hold_out:]
# X_train_scaled = X_train_scaled[:hold_out]
# y_val = train_y[hold_out:]
# y_train = train_y[:hold_out]

# val = train[hold_out:]
# train = train[:hold_out]

# X_train_scaled, train_y_t = torch.tensor(X_train_scaled.reshape(-1, 2, 200)).to(DEVICE), torch.tensor(train_y).to(DEVICE)  
# X_test_scaled = torch.tensor(X_train_scaled.reshape(-1, 2, 200)).to(DEVICE)
pca = PCA(n_components=62) # 주성분을 몇개로 할지 결정
X_train_scaled_printcipalComponents = pca.fit_transform(X_train_scaled)
X_test_scaled_printcipalComponents = pca.transform(X_test_scaled)
# principalDf = pd.DataFrame(data=printcipalComponents, columns = columns[0:62])
print(X_train_scaled_printcipalComponents.shape)

train_y_t = torch.tensor(train_y, dtype=torch.float32)
X_train_scaled = torch.tensor(X_train_scaled_printcipalComponents, dtype=torch.float32)
X_test_scaled = torch.tensor(X_test_scaled_printcipalComponents, dtype=torch.float32)

test_len = len(test_x)

N_REPEAT = 10
N_SKFOLD = 7
N_EPOCH = 100
BATCH_SIZE = 2048
VIRTUAL_BATCH_SIZE = 1024
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}

prediction = np.zeros((test_len, 1), dtype=np.float32)


## Model

In [ ]:
# https://www.kaggle.com/code/nawidsayed/lightgbm-and-cnn-3rd-place-solution/notebook
# https://www.kaggle.com/code/nagiss/9-solution-nagiss-part-2-2-weight-sharing-nn/notebook

# class Conv1d_with_SEBlock(nn.Module):
#     def __init__(self) -> None:
#         super().__init__()
#         pass
class SEBlock(nn.Module):
    def __init__(self, c, r=4):
        super().__init__()

        bottleneck = c // r
        self.squeeze = nn.AdaptiveAvgPool1d(1)
        self.excitation = nn.Sequential(
            nn.Conv1d(c, bottleneck, kernel_size=1),
            nn.SiLU(inplace=True),
            nn.Conv1d(bottleneck, c, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        bs, c, _= x.shape
        y = self.squeeze(x).view(bs, c)
        # excitation
        y = y.unsqueeze(-1)
        y = self.excitation(y).view(bs, c, 1)
        return x * y.expand_as(x)


class Conv1dBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels) -> None:
        super(Conv1dBlock, self).__init__()
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.swish = nn.SiLU()
        # self.swish = nn.SiLU(inplace=True)
        self.batchNorm1d = nn.BatchNorm1d(out_channels)
        # self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.conv1d(x)
        x = self.batchNorm1d(x)
        x = self.swish(x)
        # x = self.dropout(x)
        return x

class FcBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels) -> None:
        super(FcBlock, self).__init__()
        self.fc1 = nn.Linear(in_channels, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, out_channels)
        
        self.batchNorm1d1 = nn.BatchNorm1d(512)
        self.batchNorm1d2 = nn.BatchNorm1d(256)
        
        self.swish = nn.SiLU()
        self.dropout = nn.Dropout(0.9)

    def forward(self, x):
        x = self.fc1(x)
        x = self.batchNorm1d1(x)
        x = self.swish(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.batchNorm1d2(x)
        x = self.swish(x)
        x = self.dropout(x)
        
        x = self.fc3(x)
        return x

class SangModel(nn.Module):
    def __init__(self, input_shape):
        super(SangModel, self).__init__()
        self.input_shape = input_shape
        self.hidden_size = 64
        
        # self.conv11 = Conv1dBlock(63, self.hidden_size*1024) 
        # self.conv10 = Conv1dBlock(self.hidden_size*1024, self.hidden_size*512) 
        # self.conv9 = Conv1dBlock(self.hidden_size*512, self.hidden_size*256)  
        # self.conv8 = Conv1dBlock(63, self.hidden_size*128)  
        # self.conv7 = Conv1dBlock(63, self.hidden_size*64)  
        self.conv6 = Conv1dBlock(self.input_shape, self.hidden_size*32)  
        self.conv5 = Conv1dBlock(self.hidden_size*32, self.hidden_size*16)  
        self.conv4 = Conv1dBlock(self.hidden_size*16, self.hidden_size*8)  
        self.conv3 = Conv1dBlock(self.hidden_size*8, self.hidden_size*4)  
        self.conv2 = Conv1dBlock(self.hidden_size*4, self.hidden_size*2)  
        self.conv1 = Conv1dBlock(self.hidden_size*2, self.hidden_size)  

        self.fc = FcBlock(self.hidden_size, 1)
        
    def forward(self, x_):
        x = x_.unsqueeze(-1)
        
        # x = self.conv11(x)
        # x = self.conv10(x)
        # x = self.conv9(x)
        # x = self.conv8(x)
        # x = self.conv7(x)
        x = self.conv6(x)
        x = self.conv5(x)
        x = self.conv4(x)
        x = self.conv3(x)
        x = self.conv2(x)
        x = self.conv1(x)

        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x    

# model = SangModel(input_shape)

In [ ]:
for repeat in range(N_REPEAT):

    skf, tot = StratifiedKFold(n_splits=N_SKFOLD, random_state=repeat, shuffle=True), 0.
    for skfold, (train_idx, valid_idx) in enumerate(skf.split(X_train_scaled, train_y_t)):
        train_idx, valid_idx = list(train_idx), list(valid_idx)

        train_loader = DataLoader(TensorDataset(X_train_scaled[train_idx, :], train_y_t[train_idx]),
                                  shuffle=True, drop_last=True, **LOADER_PARAM)
        valid_loader = DataLoader(TensorDataset(X_train_scaled[valid_idx, :], train_y_t[valid_idx]),
                                  shuffle=False, drop_last=False, **LOADER_PARAM)
        test_loader = DataLoader(TensorDataset(X_test_scaled, torch.zeros((test_len,), dtype=torch.float32)),
                                 shuffle=False, drop_last=False, **LOADER_PARAM)
        
        model = SangModel(input_shape)
        # model.load_state_dict(torch.load(save_path+"model_inverse_batchNorm_repeat0_fold0_0.7506608238347163.pth")) # For overfitting. (lol)
        model.to(DEVICE)
        criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
        # criterion = torch.nn.BCELoss()
        optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=7.8e-2)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.99)
        # scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            # optimizer, T_0=N_EPOCH // 6, eta_min=4e-6)
        prediction_t, loss_t, acc_t = np.zeros((test_len, 1), dtype=np.float32), 1., 0.

        # for epoch in range(N_EPOCH):
        for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(skfold + 1, N_SKFOLD)):
            model.train()
            train_running_acc, train_running_loss, train_running_count= 0, 0., 0
            for idx, (xx, yy) in enumerate(train_loader):
                optimizer.zero_grad()
                xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                pred = model(xx).squeeze()
                loss = criterion(pred, yy)
                train_running_loss += loss.item() * len(yy)
                train_running_count += len(yy)
                train_running_acc += ((torch.sigmoid(pred) > 0.5).float() == yy).sum().item()

                loss.backward()
                optimizer.step()
                scheduler.step()
            if epoch % 5 == 0:
                print(f"Train loss: {train_running_loss / train_running_count}")
                print(f"Train accuracy: {train_running_acc / train_running_count}")
                
            with torch.no_grad():
                model.eval()
                running_acc, running_loss, running_count = 0, 0., 0
                for xx, yy in valid_loader:
                    xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                    pred = model(xx).squeeze()
                    loss = criterion(pred, yy)
                    running_loss += loss.item() * len(yy)
                    running_count += len(yy)
                    running_acc += ((torch.sigmoid(pred) > 0.5).float() == yy).sum().item()
                # print('R{:02d} S{:02d} E{:02d} | {:6.4f}, {:5.2f}%'
                #       .format(repeat + 1, skfold + 1, epoch + 1, running_loss / running_count,
                #               running_acc / running_count * 100))
                if epoch % 5 == 0:
                    print(f"Valid loss: {running_loss / running_count}")
                    print(f"Valid accuracy: {running_acc / running_count}")
                # if running_acc / running_count > acc_t:
                if running_loss / running_count < loss_t:
                    loss_t = running_loss / running_count
                    acc_t = running_acc / running_count
                    torch.save(model.state_dict(), f"{save_path}model_inverse_batchNorm_deeper64_repeat{repeat}_fold{skfold}_{loss_t}_acc{acc_t}.pth")
                    for idx, (xx, _) in enumerate(test_loader):
                        xx = xx.to(DEVICE)
                        # pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
                        # pred = (torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
                        # pred = np.float32(pred > 0.5)
                        pred = model(xx).detach().to('cpu').numpy()
                        prediction_t[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] \
                            = pred[:, :].copy()
        prediction[:, :] += prediction_t[:, :].copy() / (N_REPEAT * N_SKFOLD)
        tot += loss_t
    print('R{} -> {:6.10f}'.format(repeat + 1, tot / N_SKFOLD))

df = pd.read_csv(path+'sample_submission.csv')
df.iloc[:, 1:] = prediction


## Submission

### Using prediction in train function

In [ ]:
df.to_csv(path+'model_inverse_batchNorm_sigmoid_repeat_2.csv', index=False)

### Load and predict

In [ ]:
# model = SangModel()
model.load_state_dict(torch.load(save_path+"model_inverse_deeper1_1_repeat0_fold2_3.762843291348371e-06.pth"))
# model.to('cpu')
model.eval()

In [ ]:
# X_test_scaled = X_test_scaled.to('cpu')
# pred = model(X_test_scaled).detach().cpu().numpy()
pred = model(X_test_scaled).detach().numpy()

In [ ]:
pred

In [ ]:
submission = pd.read_csv(path+'sample_submission.csv')
submission["nerdiness"] = pred
submission.to_csv("submission_model_inverse_deeper1_1_repeat0_fold2_3.762843291348371e-06.csv", index = False)
submission